In [1]:
%load_ext autoreload
%autoreload 2

import dspy
import os
from dotenv import load_dotenv
load_dotenv()

dspy.configure(lm=dspy.OpenAI(model="gpt-4-0125-preview", max_tokens=4096))

In [2]:
from model import Repo

codebase_path = '/Users/macbook/Documents/GitHub/open-architect'
repo = Repo(root_path=codebase_path)

### Init Repo

Before Querying the repo we need to initialize the repo. During this initialization, a group of agents are going through the repo and building textual descriptions of the different componnents of the repo, these descriptions are then used by the agents to better navigate the repo.

When running for the first time on a repo, this initialization can take several minutes (21mins for open-architect) depending on the size of your repo... The descriptions are cached for the next runs

In [4]:
repo.init_repo()

### Query Repo

The query repo allows to directly ask a question to the repository. the crew is going to look for answers and output a complete description of what was found in the repo regarding your question.

In [6]:
repo_query_output = repo.query("What are the tools provided to the architect agent?")



> Entering new AgentExecutor chain...

Invoking: `Directory_expert_dev` with `{'information_request': 'List all the tools provided to the architect agent in the dev directory.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_agent_dev` with `{'information_request': 'List all the tools provided to the architect agent in the dev directory.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_architect` with `{'information_request': 'List all the tools provided to the architect agent'}`




> Entering new AgentExecutor chain...

Invoking: `Python_file_expert_run_st_architect_py` with `{'pyfile_request_question': 'List all the tools provided to the architect agent'}`


content='The tools provided to the architect agent can be found in the `run_st_architect.py` file. Here are the relevant code snippets:\n\n```python\narchitect = Architect(name="Sophia", codebase=codebase)\n```\nThis snippet creates an instance of the `Architect` class with the

In [7]:
print(repo_query_output['output'])

The tools provided to the architect agent in the `agent_dev/architect` directory are as follows:

### From `run_st_architect.py`:
1. **Architect Class Initialization**:
   - Code Snippet:
     ```python
     architect = Architect(name="Sophia", codebase=codebase)
     ```
   - Explanation: This snippet creates an instance of the `Architect` class with the name "Sophia" and the codebase loaded from a JSON file.

2. **ArchitectAgentRequest Creation**:
   - Code Snippet:
     ```python
     architectureAgentReq = ArchitectAgentRequest(
         question=prompt,
         history=[
             msg["content"]
             for msg in st.session_state.messages
         ],
     )
     ```
   - Explanation: This snippet creates an instance of the `ArchitectAgentRequest` class to send a question to the architect agent along with the history of messages exchanged between the user and the assistant.

### From `__init__.py`:
1. **Create Subtasks Tool**:
   - Description: Breaks down the user's init

### Query Repo architect

We add a architect agent to the repo. Query the architect to get a description of tasks and substacks to solve a ticket on the repo.

In [8]:
architect_output = repo.query_architect("Create a configuration to easily change the llm models used by the architect.")




> Entering new AgentExecutor chain...

Invoking: `open-architect_Repo_expert` with `{'information_request': 'Where can the llm models configuration be found in the repository?'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_src` with `{'information_request': 'Where can the llm models configuration be found in the directory?'}`




> Entering new AgentExecutor chain...

Invoking: `multi_tool_use.parallel` with `{'tool_uses': [{'recipient_name': 'functions.Python_file_expert_language_models_py', 'parameters': {'pyfile_request_question': 'Where can the llm models configuration be found in the file?'}}, {'recipient_name': 'functions.Directory_expert_lib', 'parameters': {'information_request': 'Where can the llm models configuration be found in the directory?'}}]}`


multi_tool_use.parallel is not a valid tool, try one of [Python_file_expert_language_models_py, Python_file_expert_models_py, Python_file_expert_constants_py, Directory_expert_agents, Directory_exper

BadRequestError: Error code: 400 - {'error': {'message': "'multi_tool_use.parallel' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.2.tool_calls.0.function.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
print(architect_output['output'])

To create a configuration to easily change the language models used by the architect in the `oa-small-dev` repository, you can follow these tasks and subtasks:

### Tasks:
1. **Understand the Current Setup:**
   - Review the existing setup for changing language models in the `oa-small-dev` repository.
   - Explore the `__init__.py` file in the `architect` directory to understand the key functions involved.

2. **Identify Configuration Points:**
   - Determine the specific areas in the codebase where language models are defined or referenced.
   - Identify the variables or settings that control the selection of language models.

3. **Implement Configurable Model Selection:**
   - Modify the code to introduce a configuration mechanism for selecting language models.
   - Create a function or module that allows users to easily specify the desired language model.

4. **Update Documentation:**
   - Document the new configuration options for changing language models.
   - Provide clear instru

### Visualize conv

In [10]:
output = architect_output
output

{'input': 'Create a configuration to easily change the llm models used by the architect.',
 'output': "To create a configuration to easily change the language models used by the architect in the `oa-small-dev` repository, you can follow these tasks and subtasks:\n\n### Tasks:\n1. **Understand the Current Setup:**\n   - Review the existing setup for changing language models in the `oa-small-dev` repository.\n   - Explore the `__init__.py` file in the `architect` directory to understand the key functions involved.\n\n2. **Identify Configuration Points:**\n   - Determine the specific areas in the codebase where language models are defined or referenced.\n   - Identify the variables or settings that control the selection of language models.\n\n3. **Implement Configurable Model Selection:**\n   - Modify the code to introduce a configuration mechanism for selecting language models.\n   - Create a function or module that allows users to easily specify the desired language model.\n\n4. **Updat

In [12]:
def _print_steps(steps, level=0):
    tab = """    """
    for i, (tool, step) in enumerate(steps):
        # print(tab*level + "Step", i)
        if isinstance(step, dict):
            print(tab*level + "Tool : ", tool.tool)
            print(tab*level + "Tool input : ", step['input'])
            sub_steps = step['intermediate_steps']
            _print_steps(sub_steps, level+1)
        else:
            print(tab*level + "Tool : ", tool.tool)
            # print("ai message :", step.content)



In [13]:
_print_steps(output["intermediate_steps"])

Tool :  oa-small-dev_Repo_expert
Tool input :  Explain the current setup for changing the language models used by the architect in the oa-small-dev repository.
    Tool :  Directory_expert_src
    Tool input :  Explain the setup for changing the language models used by the architect.
        Tool :  Directory_expert_agents
        Tool input :  Explain the setup for changing the language models used by the architect.
            Tool :  Directory_expert_architect
            Tool input :  Explain the setup for changing the language models used by the architect.
                Tool :  Python_file_expert___init___py


In [ ]:
repo.children[0].children

[Directory(name=agents), Directory(name=helpers)]